# [project page](https://wandb.ai/mines-paristech-cmm/fcdd-mvtec-bmvc)

i will fix some runs and add the hist avg of roc and avg precision

In [2]:
# make a cell print all the outputs instead of just the last one
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# get runs

In [3]:
WANDB_ENTITY = "mines-paristech-cmm"
WANDB_PROJECT = "fcdd-mvtec-bmvc"
WANDB_ENTITY_PROJECT = f"{WANDB_ENTITY}/{WANDB_PROJECT}"

WANDB_SWEEP_ID = None
WANDB_SWEEP_PATH = f"{WANDB_ENTITY_PROJECT}/{WANDB_SWEEP_ID}" if WANDB_SWEEP_ID else None

import wandb
api = wandb.Api()

runs = api.runs(WANDB_ENTITY_PROJECT)
# sweep = api.sweep(WANDB_SWEEP_PATH)
# runs = sweep.runs

from collections import defaultdict
lists = defaultdict(list)

def append_to_list(key, value):
    lists[key].append(value)

for run in runs: 
    append_to_list("summary", run.summary._json_dict)
    append_to_list("config", {k: v for k,v in run.config.items() if not k.startswith('_')})
    append_to_list("name", run.name)
    append_to_list("tags", run.tags)
    append_to_list("id", run.id)
    append_to_list("state", run.state)
    
import pandas as pd
runs_df = pd.DataFrame.from_dict(data=lists)

runs_df.shape
runs_df.columns

(920, 6)

Index(['summary', 'config', 'name', 'tags', 'id', 'state'], dtype='object')

In [4]:
summary_keys = sorted(set.union(*runs_df["summary"].apply(lambda x: set(x.keys())).tolist()))
"   ".join(summary_keys)

config_keys = sorted(set.union(*runs_df["config"].apply(lambda x: set(x.keys())).tolist()))
"   ".join(config_keys)

'_runtime   _step   _timestamp   _wandb   avg-precision-accumulated   batch_size_effective   epoch   loss_empirical_cdf_clip_threshold   mvtec_class_type   normal_class_fullqualified   normal_class_label   test/avg-precision   test/histogram-of-loss_maps   test/histogram-of-score_maps   test/histogram-superposed-per-class-of-loss_maps   test/histogram-superposed-per-class-of-score_maps   test/images-heatmaps-table   test/loss   test/loss-anomalous-mean   test/loss-normals-mean   test/per-instance-value-of-loss_maps   test/per-instance-value-of-score_maps   test/precision-recall-curve_table   test/roc-auc   test/roc-curve_table   test/score-anomalous-mean   test/score-mean   test/score-normals-mean   train/learning_rate_scheduler_idx=0   train/loss   train/loss-anomalous-mean   train/loss-normals-mean   train/score-anomalous-mean   train/score-mean   train/score-normals-mean   trainer/global_step   validate/avg-precision   validate/avg-precision/history-mean   validate/loss   validate/l

'batch_size   confighash_dataset_class_supervise   confighash_dataset_class_supervise_loss   confighash_dataset_class_supervise_loss_model   confighash_dataset_class_supervise_model   confighash_full   confighash_slurm   cuda_visible_devices   datadir   dataset   epochs   in_shape   it   learning_rate   lightning_accelerator   lightning_accumulate_grad_batches   lightning_check_val_every_n_epoch   lightning_deterministic   lightning_gradient_clip_algorithm   lightning_gradient_clip_val   lightning_model_summary_max_depth   lightning_ndevices   lightning_precision   lightning_profiler   lightning_strategy   logdir   loss   loss_name   lr   model   model_name   net_shape   normal_class   nparallel-runs   nworkers   optimizer   optimizer_name   pid   pin_memory   preprocessing   preview_nimages   raw_shape   real_anomaly_limit   root   scheduler   scheduler_name   scheduler_parameters   script_start_time   script_version   seed   seeds_str   slurm_array_job_id   slurm_array_task_id   slur

# filter runs

In [5]:
runs_df_filtered = runs_df

get_only_finished = runs_df['state'] == 'finished'
runs_df_filtered = runs_df_filtered[get_only_finished]

select_epoch_is_last = runs_df["summary"].apply(lambda s: s.get("epoch", None)) == 500
runs_df_filtered = runs_df_filtered[select_epoch_is_last]

runs_df_filtered.shape

(920, 6)

# make new columns

In [6]:
import numpy as np

def extract(df, from_column, key):
    return df[from_column].apply(lambda x: x.get(key, None))


def get_history(df, key):
    return 

df = runs_df_filtered

df["loss"] = extract(df, "config", "loss")
df["supervise_mode"] = extract(df, "config", "supervise_mode")
df["normal_class"] = extract(df, "config", "normal_class")
df["scheduler_parameters"] = extract(df, "config", "scheduler_parameters")

df["normal_class_fullqualified"] = extract(df, "summary", "normal_class_fullqualified")
df["loss_empirical_cdf_clip_threshold"] = extract(df, "summary", "loss_empirical_cdf_clip_threshold")

del df

runs_df_filtered.columns

Index(['summary', 'config', 'name', 'tags', 'id', 'state', 'loss',
       'supervise_mode', 'normal_class', 'scheduler_parameters',
       'normal_class_fullqualified', 'loss_empirical_cdf_clip_threshold'],
      dtype='object')

# categorical vals 

In [7]:
runs_df_filtered["loss"].unique().tolist()
runs_df_filtered["supervise_mode"].unique().tolist()
runs_df_filtered["normal_class_fullqualified"].unique().tolist()
runs_df_filtered["scheduler_parameters"].apply(tuple).unique().tolist()
runs_df_filtered["loss_empirical_cdf_clip_threshold"].unique().tolist()

['old-fcdd',
 'pixelwise-batch-avg',
 'pixelwise-batch-avg-clip-score-cdf-adaptive']

['synthetic-anomaly-confetti', 'real-anomaly']

['mvtec_14_zipper',
 'mvtec_13_wood',
 'mvtec_12_transistor',
 'mvtec_11_toothbrush',
 'mvtec_10_tile',
 'mvtec_06_leather',
 'mvtec_04_grid',
 'mvtec_09_screw',
 'mvtec_08_pill',
 'mvtec_07_metal_nut',
 'mvtec_05_hazelnut',
 'mvtec_03_carpet',
 'mvtec_02_capsule',
 'mvtec_01_cable',
 'mvtec_00_bottle']

[(0.985,), (0.9985,)]

[nan, 0.15, 0.05]

# get metric histories

In [8]:
import progressbar

df = runs_df_filtered
keys = ["validate/roc-auc", "validate/avg-precision"]

# get history of signals
hists_per_run = []
for run_id in progressbar.progressbar(df["id"], max_value=df.shape[0]):
    run = api.run(f"{WANDB_ENTITY_PROJECT}/{run_id}")
    hist = run.scan_history(keys=keys)
    hists_per_run.append([np.array([dic[key] for dic in hist]) for key in keys])

# transpose the list of lists
hists_per_run = list(map(list, zip(*hists_per_run)))


for key, list_ in zip(keys, hists_per_run):
    df[key] = list_
    
del df

 73% (678 of 920) |################      | Elapsed Time: 0:08:26 ETA:   0:03:03wandb: WARNING A graphql request initiated by the public wandb API timed out (timeout=9 sec). Create a new API with an integer timeout larger than 9, e.g., `api = wandb.Api(timeout=19)` to increase the graphql timeout.
 79% (733 of 920) |#################     | Elapsed Time: 0:09:22 ETA:   0:02:26wandb: WARNING A graphql request initiated by the public wandb API timed out (timeout=9 sec). Create a new API with an integer timeout larger than 9, e.g., `api = wandb.Api(timeout=19)` to increase the graphql timeout.
100% (920 of 920) |######################| Elapsed Time: 0:12:09 Time:  0:12:09


ValueError: Columns must be same length as key

# correct missing config of cdf loss

In [21]:
sel_cdf_loss = runs_df_filtered["loss"] == 'pixelwise-batch-avg-clip-score-cdf-adaptive'
sel_cdf_loss.sum()

sel_cdf_param_nan = runs_df_filtered["loss_empirical_cdf_clip_threshold"].isna()
sel_cdf_param_nan.sum()

sel = sel_cdf_loss & sel_cdf_param_nan
sel.sum()

runs_df_filtered.loc[sel, "loss_empirical_cdf_clip_threshold"] = 0.05

300

620

0

# check cases with more experiments than expected

In [33]:
sel_cdf_loss = runs_df_filtered["loss"] == 'pixelwise-batch-avg'
sel_cdf_loss.sum()

sel_supervise_confetti = runs_df_filtered["supervise_mode"] == 'synthetic-anomaly-confetti'
sel_supervise_confetti.sum()

sel_classes = runs_df_filtered["normal_class"].isin((10, 11))
sel_classes.sum()

sel = sel_cdf_loss & sel_supervise_confetti & sel_classes
sel.sum()

df = runs_df_filtered[sel]

df["seed"] = extract(df, "config", "seed")

# del df

320

470

140

40

/tmp/ipykernel_9338/701335707.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["seed"] = extract(df, "config", "seed")


In [37]:
df["seed"].unique().shape

(30,)

In [40]:
df[["id", "seed"]].groupby("seed").count().T

seed,2.073017e+17,5.487489e+17,1.536156e+18,1.736773e+18,2.576420e+18,2.605296e+18,2.684910e+18,3.472026e+18,4.514424e+18,4.904857e+18,...,1.160324e+19,1.209402e+19,1.323703e+19,1.449257e+19,1.502808e+19,1.581556e+19,1.698671e+19,1.706821e+19,1.757524e+19,1.834008e+19
id,1,2,1,1,2,2,1,2,2,1,...,1,1,2,1,1,1,1,1,2,2


In [56]:
df_dupes = df[["id", "seed"]].groupby("seed").apply(lambda x: sorted(set(x["id"])))
df_dupes = df_dupes[df_dupes.apply(len) > 1]
df_dupes = df_dupes.apply(lambda x: x[1:])
df_dupes

seed
5.487489e+17    [3kxv6iqk]
2.576420e+18    [3pkk6qim]
2.605296e+18    [3ay1d7l0]
3.472026e+18    [2qqxqx2d]
4.514424e+18    [kow8fxsn]
8.709818e+18    [ulrrcwkp]
1.068624e+19    [8pjieo54]
1.323703e+19    [3kh94h9f]
1.757524e+19    [2r9e4z64]
1.834008e+19    [9t01mbo7]
dtype: object

## add tag `deleteme` duplicates

In [62]:
ids_to_delete = [id_ for duped_ids in df_dupes for id_ in duped_ids]
ids_to_delete

['3kxv6iqk',
 '3pkk6qim',
 '3ay1d7l0',
 '2qqxqx2d',
 'kow8fxsn',
 'ulrrcwkp',
 '8pjieo54',
 '3kh94h9f',
 '2r9e4z64',
 '9t01mbo7']

In [66]:
import progressbar
for runid in progressbar.progressbar(ids_to_delete):
    run = api.run(f"{WANDB_ENTITY_PROJECT}/{runid}")
    run.tags = run.tags + ["deleteme", ]
    run.save()

100% (10 of 10) |########################| Elapsed Time: 0:00:04 Time:  0:00:04


i deleted them in the wandb web page

there are still 10 extra runs but i will just leave them there, this shouldn't be harmful

In [67]:
runs_df_filtered = runs_df_filtered[~runs_df_filtered["id"].isin(ids_to_delete)]

In [68]:
runs_df_filtered.shape

(910, 16)

# create histavg metrics

In [69]:
df = runs_df_filtered

metrics = ["validate/roc-auc", "validate/avg-precision"]
histavg_metrics = []

for m in metrics:
    mhistavg = f"{m}-histavg"
    histavg_metrics.append(mhistavg)
    df[mhistavg] = df[m].apply(lambda x: np.mean(x))
    
del df

/tmp/ipykernel_9338/1649431263.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[mhistavg] = df[m].apply(lambda x: np.mean(x))


In [70]:
runs_df_filtered.columns
runs_df_filtered[histavg_metrics].head()

Index(['summary', 'config', 'name', 'tags', 'id', 'state', 'loss',
       'supervise_mode', 'normal_class', 'scheduler_parameters',
       'normal_class_fullqualified', 'loss_empirical_cdf_clip_threshold',
       'validate/roc-auc', 'validate/avg-precision',
       'validate/roc-auc-histavg', 'validate/avg-precision-histavg'],
      dtype='object')

,validate/roc-auc-histavg,validate/avg-precision-histavg
0,0.512464,0.022857
1,0.684338,0.042929
2,0.839538,0.157677
3,0.899011,0.415261
4,0.873073,0.411040


In [71]:
runs_df_filtered.shape

(910, 16)

# update runs

In [73]:
import progressbar

def update_run(runid: str, df_row, metric_key: str):
    run = api.run(f"{WANDB_ENTITY_PROJECT}/{runid}")
    
    run.summary.update({
        "loss_empirical_cdf_clip_threshold": df_row["loss_empirical_cdf_clip_threshold"],
        **{m: df_row[m] for m in histavg_metrics},
    })
    run.save()

for rowidx, row in progressbar.progressbar(runs_df_filtered.iterrows(), max_value=runs_df_filtered.shape[0]):
    update_run(row['id'], row, "validate/avg-precision")

100% (910 of 910) |######################| Elapsed Time: 0:10:09 Time:  0:10:09
